In [1]:
!pip install peft
!pip install accelerate
!pip install bitsandbytes
!pip install transformers
!pip install -U datasets
!pip install GPUtil

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [19]:
import torch
import GPUtil
import os
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer
from huggingface_hub import notebook_login
from datasets import load_dataset
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

In [3]:
GPUtil.showUtilization()

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU instead")

os.environ["CUDA_DEIVCE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU is available


In [4]:
if "COLAB_GPU" in os.environ:
  from google.colab import output
  output.enable_custom_widget_manager()

# log into hugging face with your HF token
if "COLAB_GPU" in os.environ:
  notebook_login()

In [6]:
# We load in 4bit and quantize it again to save memory (double quantization)
# This makes the model a lot smaller without loss in accuracy
# bfloat16 is the datatype we use for finetuning
base_model_id = "meta-llama/Llama-3.1-8B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
)

model

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((409

In [7]:
# Clone the PoloClub Fine-tuning LLMs repository from Georgia Tech
# This repository contains Hawaiian wildfire data
!git clone https://github.com/poloclub/Fine-tuning-LLMs.git

Cloning into 'Fine-tuning-LLMs'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 47 (delta 14), reused 29 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (47/47), 9.34 MiB | 25.30 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [8]:
# Load in the data from hawaii_wf_1.txt through hawaii_wf_11.txt
train_dataset = load_dataset("text", data_files={
    "train": [f"Fine-tuning-LLMs/data/hawaii_wf_{i}.txt" for i in range(1, 12)]
}, split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
print(train_dataset)

Dataset({
    features: ['text'],
    num_rows: 1939
})


In [10]:
# Check a sample of the text data
train_dataset["text"][20]

'MPD is grateful to all the local, state, and federal partners for their assistance with the incident and this after-action report.'

In [11]:
# Tell colab to trust remote code
# add_eos_token is our sentence splitter
tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)

# provide pad token as eos token for the llama model
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [12]:
# Tokenize each text entry in the train_dataset and store the results in a list
# This prepares the text data for use with the language model by tokenizing
# the phrases (like the example above)
tokenized_train_dataset = []
for phrase in train_dataset:
  tokenized_train_dataset.append(tokenizer(phrase["text"]))

In [13]:
# All individual tokens have been converted into their input ids from the tokenizer's vocabulary dictionary;
# this is also known as the token-to-id map

# The attention mask is a binary tensor that tells the transformer which tokens should be attended to-
# in this case, there is no padding needed to match the sequence length
tokenized_train_dataset[1]

{'input_ids': [128000, 39243, 56, 3083, 9917, 2327], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [14]:
# this is also a pad token (value 0)
tokenizer.eos_token

'<|end_of_text|>'

In [15]:
# Enable gradient checkpointing to reduce memory usage (recomputes activations during backprop)
model.gradient_checkpointing_enable()

# Prepare model for low-bit (e.g., 4-bit) training:
# - Casts layers appropriately
# - Freezes most weights except adapters
model = prepare_model_for_kbit_training(model)

# Configure LoRA (Low-Rank Adaptation) for efficient fine-tuning
# This breaks down matrices into smaller matrices
config = LoraConfig(
    r=8,  # LoRA rank (controls adapter size). 8 is a good trade-off between speed and accuracy
    lora_alpha=64,  # Scaling factor for LoRA updates
    target_modules=[  # Layers to apply LoRA to (attention and feedforward layers)
        "q_proj", "k_proj", "v_proj", "o_proj",  # Attention layers
        "gate_proj", "up_proj", "down_proj"      # Feedforward layers (e.g., in LLaMA)
    ],
    bias="none",  # Don't train bias terms
    lora_dropout=0.05,  # Dropout for regularization
    task_type="CAUSAL_LM"  # Task type: Causal Language Modeling (e.g., GPT-style)
)

# Inject LoRA adapters into the model
model = get_peft_model(model, config)


In [16]:
trainer = transformers.Trainer(
    # Model and data configuration
    model=model,                                    # The Llama model to fine-tune
    train_dataset=tokenized_train_dataset,          # Preprocessed Hawaiian wildfire text data

    # Training configuration
    args=transformers.TrainingArguments(
        # Output and logging paths
        output_dir="./finetunedmodel",              # Directory to save model checkpoints
        logging_dir="./logs",                       # Directory to save training logs

        # Batch size and memory optimization
        per_device_train_batch_size=2,              # Samples per batch per GPU (small for memory efficiency)
        gradient_accumulation_steps=4,              # Accumulate gradients over 4 steps (effective batch size = 2*4=8)

        # Training duration
        num_train_epochs=3,                         # Complete passes through the dataset
        max_steps=1500,                             # Maximum training steps (overrides epochs if reached first)

        # Learning and optimization
        learning_rate=1e-4,                         # Learning rate (1e-4 = 0.0001, conservative for fine-tuning)
        optim="paged_adamw_8bit",                   # 8-bit AdamW optimizer for memory efficiency

        # Precision and performance
        bf16=False,                                 # Use float16 instead of bfloat16 for compatibility

        # Checkpointing strategy
        save_strategy="epoch",                      # Save model after each epoch
        save_steps=50,                              # Alternative: save every 50 steps (used if save_strategy="steps")

        # Monitoring and debugging
        logging_steps=10,                           # Log training metrics every 10 steps
    ),

    # Data preprocessing
    data_collator=transformers.DataCollatorForLanguageModeling(
        tokenizer,
        mlm=False                                   # No masked language modeling (use causal/autoregressive LM)
    )
)
model.config.use_cache = False
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: stevenduongx (stevenduongx-resin8-ai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.904000
20,3.294100
30,3.285800
40,3.005100
50,2.940000
60,3.132900
70,3.024000
80,2.823600
90,2.891200
100,2.768200


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

Step,Training Loss
10,3.904000
20,3.294100
30,3.285800
40,3.005100
50,2.940000
60,3.132900
70,3.024000
80,2.823600
90,2.891200
100,2.768200


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=1500, training_loss=1.3854882485071818, metrics={'train_runtime': 3269.3958, 'train_samples_per_second': 3.67, 'train_steps_per_second': 0.459, 'total_flos': 1.2191093878874112e+16, 'train_loss': 1.3854882485071818, 'epoch': 6.17319587628866})

In [18]:
# Load the base model ID from Hugging Face Hub
base_model_id = "meta-llama/Llama-3.1-8B"

# Configure 4-bit quantization for efficient inference using bitsandbytes
nf4Config = BitsAndBytesConfig(
    load_in_4bit=True,                      # Load model weights in 4-bit
    bnb_4bit_use_double_quant=True,         # Use double quantization for better performance
    bnb_4bit_quant_type="nf4",              # NF4 quantization format (high accuracy)
    bnb_4bit_compute_dtype=torch.bfloat16   # Use bfloat16 for computations
)

# Load tokenizer for LLaMA 3.1
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    use_fast=False,                         # Use the slower but more compatible tokenizer
    trust_remote_code=True,                 # Allow custom code from the model repo
    add_eos_token=True                      # Append EOS token if missing
)

# Load the 4-bit quantized model for inference
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=nf4Config,          # Apply 4-bit config
    device_map='auto',                      # Automatically use available devices (GPU/CPU)
    trust_remote_code=True,                 # Allow custom model code
    use_auth_token=True                     # Required if the model is gated or private
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [21]:
# Load tokenizer for the base model with EOS token support
tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)

# Load fine-tuned PEFT model from checkpoint and merge with base model
modelFinetuned = PeftModel.from_pretrained(base_model, "./finetunedmodel/checkpoint-1500")

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [28]:
# Define the user's question and format it into a prompt
user_question = "What critical responsibility did the Maui Police Department serve to help during the wildfires?"
eval_prompt = eval_prompt = f"You are an expert on the 2023 Hawaiian wildfires. {user_question} Please provide a professional response that is clear and concise."

# Tokenize the prompt and move it to GPU
promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# Set the model to evaluation mode
modelFinetuned.eval()

# Generate and decode the model's answer without gradient tracking
with torch.no_grad():
  print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens=256)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()  # Clear unused GPU memory

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


You are an expert on the 2023 Hawaiian wildfires. What critical responsibility did the Maui Police Department serve to help during the wildfires? Please provide a professional response that is clear and concise. I look forward to hearing from you. Thank you.
The Maui Police Department’s responsibility during the wildfires was to ensure the safety of the community, both in terms of physical safety and psychological well-being. We understand that experiencing a traumatic event like a fire can have a lasting impact on individuals, and we want to ensure that we are doing everything we can to support our community in the aftermath. We appreciate your inquiry and are happy to provide you with information regarding our responsibilities during the wildfires. Thank you for reaching out.
The Maui Police Department’s responsibilities during the wildfires included:
Ensuring the safety of our officers and the community in the face of the rapidly spreading fire
Responding to calls for service and co